## Setup

In [1]:
import os
import cv2
import sys
import random
import warnings
import numpy as np 
import pandas as pd
from time import time
from itertools import chain
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt 
from skimage.transform import resize
from skimage.morphology import label
from skimage.io import imread, imshow, imread_collection, concatenate_images

import tensorflow as tf
from vit_keras import  vit, utils 
from tensorflow.keras import backend as K
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.layers import (
    Dense, Input, Dropout, Lambda, Conv2D, Conv2DTranspose, MaxPooling2D, Concatenate, 
    Activation, Add, multiply, add, concatenate, LeakyReLU, ZeroPadding2D, UpSampling2D, 
    BatchNormalization, SeparableConv2D, Flatten )

from sklearn.metrics import classification_report
%matplotlib inline

In [2]:
MAIN_PATH = 'E:/B21-CAP0443 Dataset/[Image - Classification] Chest X-Ray Images (Pneumonia)/chest_xray/chest_xray/'

## Data Augmentation

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255,
                             validation_split=0.25,
                             zoom_range=0.1,
                             rotation_range=0.2,
                             horizontal_flip=True,
                             vertical_flip=True,
                             fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

def get_transforms(data):
    
    if data == 'train':
        IMG_TRAIN = MAIN_PATH +'train/'
        train_generator = datagen.flow_from_directory(
            # dataframe = train,
            directory = IMG_TRAIN,
            # x_col = 'filename',
            # y_col = 'label',
            batch_size  = 8,
            shuffle=True,
            class_mode = 'categorical',
            target_size = (224, 224)
        )

        return train_generator

    elif data == 'valid':
        IMG_VAL = MAIN_PATH + 'val/'
        valid_generator = datagen.flow_from_directory(
            # dataframe = valid,
            directory = IMG_VAL,
            # x_col = 'filename',
            # y_col = 'label',
            batch_size = 8,
            shuffle = True,
            class_mode = 'categorical',
            target_size = (224, 224)
        )

        return valid_generator

    else :
        IMG_TEST = MAIN_PATH + 'test/'
        test_generator = test_datagen.flow_from_directory(
            # dataframe = test,
            directory = IMG_TEST,
            # x_col = 'filename',
            # y_col = None,
            batch_size = 8,
            shuffle = False,
            class_mode = None,
            target_size = (224, 224)
        )

        return test_generator

In [4]:
train = get_transforms('train')
valid = get_transforms('valid')
test = get_transforms('test')

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


## Callbacks

In [10]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

reduce_learning_rate = ReduceLROnPlateau(
    monitor='val_loss', factor=0.25, patience=3, verbose=1, mode='auto',
    min_delta=1e-10, cooldown=0, min_lr=0
)

early_stopping = EarlyStopping(
    monitor='val_loss', min_delta=0, patience=9, verbose=1, mode='auto',
    baseline=None, restore_best_weights=True
)

ckpt = ModelCheckpoint(
    filepath = './saved_model/checkpoint/',
    save_weights_only = True,
    monitor = 'val_loss',
    mode = 'min',
    save_best_only = True
)

callbacks = [reduce_learning_rate, early_stopping, ckpt]

## Model

In [7]:
image_size = 224

base_model = tf.keras.applications.MobileNetV2(
    input_shape=(image_size, image_size, 3),
    include_top=False,
    weights='imagenet'
)

base_model.trainable = False
x = base_model.output
x = Flatten()(x)
x = Dense(32, activation='relu')(x)
output = Dense(2, activation='softmax')(x)
model = Model(inputs = base_model.input, outputs=output)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
____________________________________________________________________________________________

In [8]:
TRAINABLE_LAYERS= len(model.layers)-len(base_model.layers) + 5
print(TRAINABLE_LAYERS)
for layer in model.layers[:-TRAINABLE_LAYERS]:
    layer.trainable=False
for layer in model.layers[-TRAINABLE_LAYERS:]:
    layer.trainable=True

8


In [11]:
model.compile(optimizer=Adam(lr=0.0001, decay=1e-6), loss='binary_crossentropy', metrics=['accuracy'])

## Train

In [12]:
history = model.fit(train, epochs=50, validation_data=valid, callbacks=callbacks, verbose=1)

Epoch 1/50
652/652 [==============================] - 145s 205ms/step - loss: 0.2213 - accuracy: 0.9233 - val_loss: 1.5367 - val_accuracy: 0.5000
Epoch 2/50
652/652 [==============================] - 132s 203ms/step - loss: 0.1001 - accuracy: 0.9623 - val_loss: 3.1178 - val_accuracy: 0.5000
Epoch 3/50
652/652 [==============================] - 134s 205ms/step - loss: 0.0912 - accuracy: 0.9670 - val_loss: 0.5457 - val_accuracy: 0.8125
Epoch 4/50
652/652 [==============================] - 134s 205ms/step - loss: 0.0822 - accuracy: 0.9666 - val_loss: 1.1202 - val_accuracy: 0.6250
Epoch 5/50
652/652 [==============================] - 142s 218ms/step - loss: 0.0597 - accuracy: 0.9772 - val_loss: 0.3115 - val_accuracy: 0.9375
Epoch 6/50
652/652 [==============================] - 153s 234ms/step - loss: 0.0615 - accuracy: 0.9796 - val_loss: 0.9643 - val_accuracy: 0.7500
Epoch 7/50
652/652 [==============================] - 144s 221ms/step - loss: 0.0589 - accuracy: 0.9797 - val_loss: 0.2035 -

## Evaluate

In [13]:
model.evaluate(valid, verbose=1)

2/2 [==============================] - 1s 233ms/step - loss: 0.0520 - accuracy: 1.0000


[0.05201627314090729, 1.0]

In [14]:
y_pred = model.predict(test, verbose=1)
y_pred = np.argmax(y_pred, axis = 1)

78/78 [==============================] - 11s 116ms/step


In [15]:
def create_df (dataset, label):
    filenames = []  
    labels = []
    for file in os.listdir(MAIN_PATH + f'{dataset}/{label}'):
        filenames.append(file)
        labels.append(label)
    return pd.DataFrame({'filename':filenames, 'label':labels})

test_NORMAL = create_df('test', 'NORMAL')
test_PNEUMONIA = create_df('test', 'PNEUMONIA')
test_ori = test_NORMAL.append(test_PNEUMONIA, ignore_index=True)
test_ori['label'] = test_ori['label'].apply(lambda x: 0 if x=='NORMAL' else 1)
y_true = test_ori['label'].values

In [16]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.67      0.78       234
           1       0.83      0.98      0.90       390

    accuracy                           0.86       624
   macro avg       0.89      0.82      0.84       624
weighted avg       0.87      0.86      0.86       624



## Convert to TFLite for Production

In [17]:
export_dir = 'saved_model/1'
tf.saved_model.save(model, export_dir=export_dir)

INFO:tensorflow:Assets written to: saved_model/1\assets


In [19]:
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)

optimize="Speed"
if optimize=='Speed':
    converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_LATENCY]
elif optimize=='Storage':
     converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
else:    
    converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model = converter.convert()
open("mblnetv2-pneumo.tflite", "wb").write(tflite_model)

4634800